In [71]:
import pandas as pd
import gc
import re
!pip install icd-mappings
from icdmappings import Mapper
import pandas as pd
from sklearn.impute import KNNImputer
from google.colab import drive

In [50]:
# Load Diagnosis Dataset in chunks (all CPU-based)
drive.mount('/content/drive', force_remount=True)
file_path = "/content/drive/MyDrive/MIMIC/mimic-iv-ed-2.2/ed/diagnosis.csv"
chunk_size = 100000  # Adjust based on memory
chunks = []

Mounted at /content/drive


In [51]:
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    chunk["icd_code"] = chunk["icd_code"].astype("category")
    chunk["icd_version"] = chunk["icd_version"].astype("int32")
    chunks.append(chunk)

In [52]:
diagnosis = pd.concat(chunks, ignore_index=True)
del chunks
gc.collect()
diagnosis = diagnosis[diagnosis["seq_num"] == 1]

In [53]:
print("Diagnosis dataset loaded using CPU.")

Diagnosis dataset loaded using CPU.


In [54]:
print(diagnosis.head(20))

    subject_id   stay_id  seq_num icd_code  icd_version  \
0     10000032  32952584        1     4589            9   
3     10000032  33258284        1     5728            9   
7     10000032  35968195        1     5715            9   
10    10000032  38112554        1    78959            9   
14    10000032  39399961        1    78097            9   
16    10000084  35203156        1     R531           10   
18    10000084  36954971        1    R4182           10   
20    10000108  32522732        1     7822            9   
21    10000108  36533795        1     5283            9   
22    10000108  39513268        1     5283            9   
23    10000115  30295111        1  S0181XD           10   
25    10000115  38081480        1  S025XXA           10   
28    10000117  30632130        1  S098XXA           10   
31    10000117  32642808        1    R1310           10   
32    10000117  33176849        1  S72092A           10   
34    10000178  31721172        1     M109           10 

In [55]:
# Install and use icd-mappings
mapper = Mapper()

In [56]:
# Separate ICD9 and ICD10 codes
diagnosis["icd_code_str"] = diagnosis["icd_code"].astype(str)
icd9_mask = diagnosis["icd_version"] == 9
icd10_mask = diagnosis["icd_version"] == 10

In [57]:
# Map ICD9 → ICD10
diagnosis["icd10_mapped"] = None
diagnosis.loc[icd9_mask, "icd10_mapped"] = mapper.map(
    diagnosis.loc[icd9_mask, "icd_code_str"].tolist(),
    source='icd9',
    target='icd10'
)

In [58]:
# Map ICD10 (native and mapped) to chapters
diagnosis["chapter"] = None

In [59]:
# ICD10 native
diagnosis.loc[icd10_mask, "chapter"] = mapper.map(
    diagnosis.loc[icd10_mask, "icd_code_str"].tolist(),
    source='icd10',
    target='chapter'
)

In [60]:
# ICD9 converted to ICD10 → chapter
diagnosis.loc[icd9_mask, "chapter"] = mapper.map(
    diagnosis.loc[icd9_mask, "icd10_mapped"],
    source='icd10',
    target='chapter'
)

In [61]:
# Define chapter number to text mapping
chapter_mapping = {
    1:  "Certain infectious and parasitic diseases",
    2:  "Neoplasms",
    3:  "Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism",
    4:  "Endocrine, nutritional and metabolic diseases",
    5:  "Mental and behavioural disorders",
    6:  "Diseases of the nervous system",
    7:  "Diseases of the eye and adnexa",
    8:  "Diseases of the ear and mastoid process",
    9:  "Diseases of the circulatory system",
    10: "Diseases of the respiratory system",
    11: "Diseases of the digestive system",
    12: "Diseases of the skin and subcutaneous tissue",
    13: "Diseases of the musculoskeletal system and connective tissue",
    14: "Diseases of the genitourinary system",
    15: "Pregnancy, childbirth and the puerperium",
    16: "Certain conditions originating in the perinatal period",
    17: "Congenital malformations, deformations and chromosomal abnormalities",
    18: "Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified",
    19: "Injury, poisoning and certain other consequences of external causes",
    20: "External causes of morbidity and mortality",
    21: "Factors influencing health status and contact with health services"
}

# Apply the mapping
diagnosis["chapter_text"] = (
    diagnosis["chapter"]
    .replace("None", pd.NA)     # Just in case string "None" exists
    .astype("float")            # Convert to float first (safe for missing)
    .astype("Int64")            # Nullable integer type
    .map(chapter_mapping)
)

In [64]:
# Load ICD-10 Supercategory Descriptions
icd10_super_df = pd.read_csv(
    "/content/drive/MyDrive/MIMIC/icd10codes.txt",
    sep="\t",
    usecols=["ICD-10-CM CODE", "Default CCSR CATEGORY DESCRIPTION IP"]
)
icd10_super_df.columns = ["icd_code", "dx_super"]
icd10_super_dict = {row["icd_code"].strip(): row["dx_super"] for _, row in icd10_super_df.iterrows()}

# Load ICD-9 mappings from icd9codes.txt
icd9_mapping = {}
with open("/content/drive/MyDrive/MIMIC/icd9codes.txt", "r") as f:
    category = None
    for line in f:
        line = line.strip()
        if re.match(r"^\d+\s{2,}", line):
            category = line.split(maxsplit=1)[1]
        elif category and line:
            codes = line.split()
            icd9_mapping.update({code.strip(): category for code in codes})
print("✅ ICD-9 CCSR mapping loaded.")

# Standardize ICD code formatting
diagnosis["icd_code"] = diagnosis["icd_code"].astype(str).str.strip().str.upper()

# Correct misclassified ICD-10 codes
icd10_misclassified = ["J45901", "O8612", "S01312A", "W01190A", "R270"]
diagnosis.loc[
    (diagnosis["icd_code"].isin(icd10_misclassified)) & (diagnosis["icd_version"] == 9),
    "icd_version"
] = 10
print("✅ Misclassified ICD-10 codes corrected.")

# Map codes to supercategories
diagnosis["dx_super"] = diagnosis["icd_code"].map(icd10_super_dict)
diagnosis["dx_super"] = diagnosis["dx_super"].fillna(diagnosis["icd_code"].map(icd9_mapping))

print("✅ Supercategory descriptions added to diagnosis.")

✅ ICD-9 CCSR mapping loaded.
✅ Misclassified ICD-10 codes corrected.
✅ Supercategory descriptions added to diagnosis.


In [65]:
# Show example rows
print(diagnosis.head(20))

    subject_id   stay_id  seq_num icd_code  icd_version  \
0     10000032  32952584        1     4589            9   
3     10000032  33258284        1     5728            9   
7     10000032  35968195        1     5715            9   
10    10000032  38112554        1    78959            9   
14    10000032  39399961        1    78097            9   
16    10000084  35203156        1     R531           10   
18    10000084  36954971        1    R4182           10   
20    10000108  32522732        1     7822            9   
21    10000108  36533795        1     5283            9   
22    10000108  39513268        1     5283            9   
23    10000115  30295111        1  S0181XD           10   
25    10000115  38081480        1  S025XXA           10   
28    10000117  30632130        1  S098XXA           10   
31    10000117  32642808        1    R1310           10   
32    10000117  33176849        1  S72092A           10   
34    10000178  31721172        1     M109           10 

In [68]:
# 🔹 1. Load the EDstays table
edstays_path = "/content/drive/MyDrive/MIMIC/mimic-iv-ed-2.2/ed/edstays.csv"
edstays = pd.read_csv(edstays_path, parse_dates=["intime", "outtime"])
print("✅ EDstays table loaded.")

# 🔹 2. Check for missing values
print("\n🔍 Missing values in EDstays:")
missing = edstays.isnull().sum()
total = len(edstays)
missing_summary = pd.DataFrame({
    'Missing (n)': missing,
    'Missing (%)': (missing / total * 100).round(2)
})
print(missing_summary)

# 🔹 3. Calculate length of stay in seconds
edstays["length_of_stay"] = (edstays["outtime"] - edstays["intime"]).dt.total_seconds()
print("\n⏱️ Length of stay column added.")

# 🔹 4. Merge with diagnosis table on 'stay_id' (assumes diagnosis is already loaded)
original_rows = len(edstays)
edstays = edstays.merge(diagnosis, on="stay_id", how="left")
merged_rows = len(edstays)

print(f"\n🔗 Merge complete. Rows before: {original_rows}, after: {merged_rows}")
if merged_rows < original_rows:
    print("⚠️ Warning: Some rows were lost in the merge.")
elif merged_rows > original_rows:
    print("⚠️ Extra rows added due to multiple matches (one-to-many join).")
else:
    print("✅ No rows lost during merge.")



✅ EDstays table loaded.

🔍 Missing values in EDstays:
                   Missing (n)  Missing (%)
subject_id                   0         0.00
hadm_id                 222071        52.24
stay_id                      0         0.00
intime                       0         0.00
outtime                      0         0.00
gender                       0         0.00
race                         0         0.00
arrival_transport            0         0.00
disposition                  0         0.00

⏱️ Length of stay column added.

🔗 Merge complete. Rows before: 425087, after: 425087
✅ No rows lost during merge.


In [69]:
print(edstays.head(20))

    subject_id_x     hadm_id   stay_id              intime  \
0       10000032  22595853.0  33258284 2180-05-06 19:17:00   
1       10000032  22841357.0  38112554 2180-06-26 15:54:00   
2       10000032  25742920.0  35968195 2180-08-05 20:58:00   
3       10000032  29079034.0  32952584 2180-07-22 16:24:00   
4       10000032  29079034.0  39399961 2180-07-23 05:54:00   
5       10000084  23052089.0  35203156 2160-11-20 20:36:00   
6       10000084  29888819.0  36954971 2160-12-27 18:32:00   
7       10000108  27250926.0  36533795 2163-09-27 16:18:00   
8       10000108         NaN  32522732 2163-09-16 16:34:00   
9       10000108         NaN  39513268 2163-09-24 16:14:00   
10      10000115         NaN  30295111 2154-12-17 16:37:00   
11      10000115         NaN  38081480 2154-12-10 02:04:00   
12      10000117  22927623.0  32642808 2181-11-14 21:51:00   
13      10000117  27988844.0  33176849 2183-09-18 08:41:00   
14      10000117         NaN  30632130 2183-07-17 10:30:00   
15      

In [ ]:
# Load triage table
triage = pd.read_csv("/content/drive/MyDrive/MIMIC/mimic-iv-ed-2.2/ed/triage.csv")

# Drop 'pain' column
triage = triage.drop(columns=["pain"])

# Define vital sign columns and outlier thresholds
vital_signs_columns = ['temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp']
outlier_ranges = {
    'temperature': (82.4, 109.4),
    'heartrate': (25, 225),
    'resprate': (6, 50),
    'o2sat': (60, 100),
    'sbp': (60, 250),
    'dbp': (30, 150),
}

# Convert Celsius to Fahrenheit only when plausible
def convert_celsius_to_fahrenheit(temp):
    if pd.notna(temp) and 28 <= temp <= 43:
        return (temp * 9 / 5) + 32
    return temp

triage['temperature'] = triage['temperature'].apply(convert_celsius_to_fahrenheit)

# Remove outliers based on the defined thresholds
for col, (low, high) in outlier_ranges.items():
    triage.loc[(triage[col] < low) | (triage[col] > high), col] = pd.NA

# Keep a copy to track imputation
triage_before_imputation = triage.copy()

# Apply KNN imputation
imputer = KNNImputer(n_neighbors=5)
triage[vital_signs_columns] = imputer.fit_transform(triage[vital_signs_columns])

# Count total number of values imputed
imputed_mask = triage_before_imputation[vital_signs_columns].isnull() & triage[vital_signs_columns].notnull()
imputed_total = imputed_mask.sum().sum()
print(f"Total values imputed by KNN: {imputed_total}")

# Join triage to edstays using a left join on stay_id
pre_merge_rows = edstays.shape[0]
edstays = edstays.merge(triage, on="stay_id", how="left")
post_merge_rows = edstays.shape[0]

print(f"Join complete. Rows before merge: {pre_merge_rows}, after merge: {post_merge_rows}")
nulls_after_merge = edstays.isnull().sum()
print("Null values by column after merge:")
print(nulls_after_merge[nulls_after_merge > 0])

In [ ]:
print(edstays.head(20))

In [ ]:
# Load vitalsign table
vitalsign_path = "/content/drive/MyDrive/MIMIC/mimic-iv-ed-2.2/ed/vitalsign.csv"
vitalsign = pd.read_csv(vitalsign_path)

print(f"Vitalsign data loaded: {vitalsign.shape[0]} rows")

# Select relevant columns from edstays
columns_to_add = ['stay_id', 'chapter', 'chapter_text', 'dx_super']
edstays_subset = edstays[columns_to_add].drop_duplicates()

# Join with vitalsign on stay_id
pre_merge_rows = vitalsign.shape[0]
vitalsign = vitalsign.merge(edstays_subset, on="stay_id", how="left")
post_merge_rows = vitalsign.shape[0]

print(f"Join complete. Rows before: {pre_merge_rows}, after: {post_merge_rows}")
nulls = vitalsign[["chapter", "chapter_text", "dx_super"]].isnull().sum()
print("Nulls in joined columns:")
print(nulls[nulls > 0])

print(vitalsign.head(20))